In [ ]:
from pdf2image import convert_from_path

In [ ]:
import io
import PIL
import gridfs
import pymongo as py
from ipywidgets import interact
import IPython.display as ipy


def browse_pages(d):
    n = d.n_frames
    def view_image(i):
        d.seek(i)
        img_bytes = io.BytesIO()
        d.save(img_bytes, format='png')
        img_bytes = img_bytes.getvalue()
        ipy.display(ipy.Image(img_bytes))
    interact(view_image, i=(0,n-1))

    
def get_pages(c, d, u):
    client = py.MongoClient(c)
    db = client[d]
    fs = gridfs.GridFS(db)
    tiff_bytes = io.BytesIO()

    doc = db.aug_meta.find_one({"uuid": u})
    a = PIL.Image.open(fs.get(doc['page_images'][0]))
    imgs = [PIL.Image.open(fs.get(x)) for x in doc['page_images'][1:]]
    a.save('test.tiff', save_all=True, append_images=imgs)
    p = PIL.Image.open('test.tiff')
    return p

In [ ]:
docs = dict()
#content_type = "application/msword"
#content_type = "application/vnd.ms-powerpoint"
content_type = "application/vnd.ms-excel"

for i, uuid in enumerate(
    py.MongoClient('mongo')['docs']['aug_meta'].find({"$and": [
        {"Content-Type.Content": content_type},
        {"raw_file": {"$exists": True}}]}, {'_id': False, 'uuid': True})):
    docs[i] = uuid['uuid']
len(docs)

In [ ]:
h = get_pages("mongo", "docs", docs[52])
browse_pages(h)